# Prepare Superresolution Training Data with eo-learn

There are many examples and resources for training superresolution networks on (satellite) imagery:
- [MDL4EO](https://mdl4eo.irstea.fr/2019/03/29/enhancement-of-sentinel-2-images-at-1-5m/)
- [ElementAI HighRes-Net](https://github.com/ElementAI/HighRes-net)
- [Fast.ai superresolution](https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson7-superres.ipynb)

We'll show you how to use `eo-learn` to prepare data for these tasks (and an example of training the network with `fastai`)

First you'll need to download the [Spacenet Challenge: Paris Data](https://spacenetchallenge.github.io/AOI_Lists/AOI_3_Paris.html). We're using this to get high resolution image chips.

In [1]:
from os import path as op
from glob import glob
import datetime

from eolearn.io import ImportFromTiff, S2L2AWCSInput
from eolearn.core import FeatureType, LinearWorkflow, EOTask
from sentinelhub import BBox, CRS

from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
spacenet_images = glob('AOI_3_Paris_Train/RGB-PanSharpen/*.tif')

In [3]:
# Import the Spacenet chips into EOPatches, as a feature called "spacenet"
input_task = ImportFromTiff((FeatureType.DATA_TIMELESS, 'spacenet'))

In [4]:
# Add Sentinel 2 L2A to our EOPatches covering the same area
time_interval = ('2017-02-28', '2017-04-01') # roughly matching the spacenet dates
layer = 'TRUE-COLOR-S2-L2A'

add_l1c = S2L2AWCSInput(layer=layer, resx='10m', resy='10m', maxcc=0.1, time_difference=datetime.timedelta(hours=2))

In [5]:
# Save the Spacenet and Sentinel images in separate folders. Resize our images when saving 

BIG_SIZE = (256, 256)
SMALL_SIZE = (64, 64)
INPUT_FOLDER = 'input'
TARGET_FOLDER = 'target'

class CustomSave(EOTask):
    def execute(self, eopatch, image_name=None):
        # if we don't have enough data, don't save
        spacenet_array = eopatch.data_timeless['spacenet']
        data_pct = (np.count_nonzero(spacenet_array) / spacenet_array.size)
        if data_pct < 0.9:
            return eopatch
           
        # resize images, rescale to 8bit
        sentinel_array = eopatch.data[layer][0]
        sentinel_array_8bit = (sentinel_array * 255.).astype(np.uint8)
        sentinel_img = Image.fromarray(sentinel_array_8bit).resize(SMALL_SIZE, resample=Image.BILINEAR)
        sentinel_img.save(op.join(INPUT_FOLDER, f'{image_name}.png'))
        
        spacenet_array_8bit = ((spacenet_array - np.min(spacenet_array, axis=(0, 1))) / (np.max(spacenet_array, axis=(0, 1)) - np.min(spacenet_array, axis=(0, 1))) * 255).astype(np.uint8)
        spacenet_image = Image.fromarray(spacenet_array_8bit).resize(BIG_SIZE, resample=Image.BILINEAR)
        spacenet_image.save(op.join(TARGET_FOLDER, f'{image_name}.png'))
        
        return eopatch
    
custom_save = CustomSave()

In [6]:
# Create this as a EOWorkflow to run over all the images
prepare_data = LinearWorkflow(
    input_task,
    add_l1c,
    custom_save
)

In [7]:
# Execute the workflow
pbar = tqdm(total=len(spacenet_images))
for image in spacenet_images:
    image_name = op.splitext(op.basename(image))[0].replace('RGB-PanSharpen_AOI_3_Paris_', '')
    workflow_input = {
        input_task: dict(filename=image),
        add_l1c: dict(time_interval=time_interval),
        custom_save: dict(image_name=image_name)
    }
    prepare_data.execute(workflow_input)
    pbar.update(1)
